## קוד מבוא

In [241]:
import os
import pathlib
import pandas as pd
import geopandas as gpd
import fiona

In [242]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## פונקציות

In [243]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [244]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [245]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [246]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [247]:
def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
    code_to_find=list(df_to_geoode[unique_field].unique())

    gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

    gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

    gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

    return gpd_for_geocode_in_one_taz

In [248]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df


In [249]:
def print_before(name):
    return print("{} before: ".format(name),round(forecast['{}'.format(name)].sum(),0))

In [250]:
def print_after(name):
    return print("{} after: ".format(name),round(forecast['{}'.format(name)].sum(),0))

In [251]:
def logic_test_for_forecast(taz):
    
    
    print ('taz num under 0:',list(taz.loc[taz['TAZ']<=0]['TAZ']))
    
    print ('taz num duplicate:',list(taz.loc[taz.duplicated(subset='TAZ',keep=False)]['TAZ']))
    
    print ('yosh_unique:',list(taz.yosh.unique()))
    
    print ('in_jerusalem_metropolin_unique:',list(taz.in_jerusalem_metropolin.unique()))
    
    print ('jerusalem_city_unique:',list(taz.jerusalem_city.unique()))
    
    print ('sector_unique:',list(taz.sector.unique()))
    


    col=['hh_total',
    'pop',
    'age0_4',
    'age5_9',
    'age10_14',
    'age15_19',
    'age20_24',
    'age25_29',
    'age30_34',
    'age35_39',
    'age40_44',
    'age45_49',
    'age50_54',
    'age55_59',
    'age60_64',
    'age65_69',
    'age70_74',
    'age75up',
    'emp_tot',
    'indus',
    'com_hotel',
    'business',
    'public',
    'education',
    'agri',
    'student',
    'univ',
    'UO_Hi_Ed',
    'pop_emp_employed']

    taz_to_check_minus=[]
    col_to_check_minus=[]
    for i in col:
        taz_to_check_minus=taz_to_check_minus+list(taz.loc[taz['{}'.format(i)]<0]['TAZ'])
    
    
    print ('taz num with minus:',list(set(taz_to_check_minus)))
    
    print ('pop more then hh:',list(taz.loc[taz['sector']!="Palestinian"].loc[~(taz['hh_total']<=taz['pop'])]['TAZ']))
    
    print ('hh missing pop:',list(taz.loc[taz['sector']!="Palestinian"].loc[taz['hh_total']>0].loc[taz['pop']<=0]['TAZ']))
    
    print ('pop missing hh:',list(taz.loc[taz['sector']!="Palestinian"].loc[taz['pop']>0].loc[taz['hh_total']<=0]['TAZ']))



    col=['age0_4',
    'age5_9',
    'age10_14',
    'age15_19',
    'age20_24',
    'age25_29',
    'age30_34',
    'age35_39',
    'age40_44',
    'age45_49',
    'age50_54',
    'age55_59',
    'age60_64',
    'age65_69',
    'age70_74',
    'age75up',]

    taz['pop_check']=taz[col].sum(axis=1)-taz['pop']
    
    taz['pop_check']=taz['pop_check'].round(0)
    
    print ('pop vs age dis Mistake:',list(taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_check']!=0]['TAZ']))   

    

    col=[
    'age0_4',
    'age5_9',
    'age10_14',
    'age15_19',
    'age20_24',
    'age25_29',
    'age30_34',
    'age35_39',
    'age40_44',
    'age45_49',
    'age50_54',
    'age55_59',
    'age60_64',
    'age65_69',
    'age70_74',
    'age75up']

    taz_to_check_for_age=[]
    for i in col:
        taz_to_check_for_age=taz_to_check_for_age+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['pop']<=0]['TAZ'])
        taz_to_check_for_age=taz_to_check_for_age+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['hh_total']<=0]['TAZ'])
        
    print ('taz num with age dis prob:',taz_to_check_for_age)

    col=['indus',
    'com_hotel',
    'business',
    'public',
    'education',
    'agri']

    taz['emp_check']=taz[col].sum(axis=1)-taz['emp_tot']
    taz['emp_check']=abs(taz['emp_check'].round(0))
    
    taz_prob_emp_total=list(taz.loc[taz['emp_check']>1]['TAZ'])
    
    print ('taz num with emp total prob:',taz_prob_emp_total)

    

    taz_to_check_for_split_emp=[]
    
    for i in col:
        taz_to_check_for_split_emp=taz_to_check_for_split_emp+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['emp_tot']<=0]['TAZ'])

        
    print ('taz num with emp dis prob:',taz_to_check_for_split_emp)
    

    print ('taz num with pop_emp_employed worng :',list(taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_emp_employed']>0].loc[taz['pop']<=0]['TAZ']))
    
    return

In [252]:
def find_files_with_pattern(folder_path, pattern):
    """
    Find files in a directory that match a certain pattern.
    
    Args:
    - directory (str): The directory path.
    - pattern (str): The pattern to search for in file names.
    
    Returns:
    - List of file paths matching the pattern.
    """
    files = []
    for root, _, filenames in os.walk(folder_path):
        for filename in filenames:
            if pattern in filename:
                files.append(os.path.join(root, filename))
    return files

In [253]:
def drop_geo(geoDF):
    geoDF = geoDF.drop(columns='geometry')
    return geoDF

In [254]:
#העלת משתנים להרצת הקוד
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

##  העלת טבלאות אוכלוסייה

In [255]:
matching_files=find_files_with_pattern(r'{}\Intermediates\change_pop'.format(software_data_folder_location), 'pop_2025_bau_changed')
file = pathlib.Path(matching_files[0])
v_date = file.name[:6]
folder = str(file.parent)
v_date

'240318'

In [256]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

year=['2025','2030','2035','2040','2045','2050']

for y in year: 
    
    col=['Taz_num',
 'aprt',
 'pop',
 'pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']
    
    folder_path=r'{}\Intermediates\change_pop'.format(software_data_folder_location)

    file_name='{}_pop_{}_bau_changed'.format(v_date,y)

    locals()['forecast_pop_{}_bau'.format(y)]=up_load_df(folder_path,file_name)[col]

## העלת טבלת מצב קיים

In [257]:
matching_files=find_files_with_pattern(r'{}'.format(forecast_version_folder_location), '2020_jtmt_forcast_full')

file = pathlib.Path(matching_files[0])
file_name = file.name
folder = str(file.parent)

In [258]:
col=['Taz_num',
     'Taz_name',
     'Muni_Heb',
     'jeru_metro',
     'zonetype',
     'in_jerusal',
     'main_secto',
     'aprt_20',
     'student_dorms',
     'student_yeshiva_with_add_from_old',
     'student',
     'uni_students','emp_from_uni_student','pop','emp_from_Yeshiva_student','yeshiva_dorms_pop_15',
     'yeshiva_dorms_pop_20',
     'yeshiva_dorms_pop_25',
     'yeshiva_dorms_pop_sum',]

new_column_names = {'aprt_20': 'aprt','student_yeshiva_with_add_from_old':'student_yeshiva'}

forecast_2020=up_load_df(forecast_version_folder_location,file_name[:-5])[col]
forecast_2020=forecast_2020.rename(columns=new_column_names)

In [259]:
forecast_2020['jew']=0

In [260]:
forecast_2020.loc[(forecast_2020['main_secto']=="U_Orthodox")|(forecast_2020['main_secto']=="Jewish"),'jew']=1

In [261]:
folder_path=r'{}\background_files'.format(software_data_folder_location)

taz_sector_change=up_load_df(folder_path,'230704_taz_num_convert_from_jewish_to_u_orthodox')

lst_change_sector=list(taz_sector_change.Taz_num)

forecast_2020.loc[(forecast_2020['Taz_num'].isin(lst_change_sector)),'main_secto']='U_Orthodox'

## העלת שכבת לא עוקב

In [262]:
EMP_kibolet=up_load_gdb(r'{}\background_files\EMP_KIBOLET.gdb'.format(software_data_folder_location),'EMP_kibolet')

EMP_kibolet=EMP_kibolet.fillna(0)

EMP_kibolet['emp_kayim_no_palestinians']=EMP_kibolet['kayim_emp']-EMP_kibolet['Palestinians']

EMP_kibolet['emp_current']=EMP_kibolet['emp_kayim_no_palestinians']

In [263]:
col=[ 'F2025',
 'F2030',
 'F2035',
 'F2040',
 'F2045',
 'F2050']

In [264]:
EMP_kibolet[col]=EMP_kibolet[col].apply(lambda x: x * (EMP_kibolet['kibolt']-EMP_kibolet['kayim_emp']))

In [265]:
EMP_kibolet[col]=EMP_kibolet[col].cumsum(axis=1)

In [266]:
EMP_kibolet[col]=EMP_kibolet[col].apply(lambda x: x +EMP_kibolet['emp_kayim_no_palestinians'])

In [267]:
emp_not_okev_current=EMP_kibolet['emp_kayim_no_palestinians'].sum().item()

## אזורי תעסוקה עיבודים והצלבה עם אזורי תנועה

In [268]:
taz=up_load_shp(r'{}\background_files\TAZ_V4_230518_Published.shp'.format(software_data_folder_location))

taz['taz_area']=taz.area

EMP_kibolet['emp_area']=EMP_kibolet.area

EMP_kibolet['ID']=EMP_kibolet.index

In [269]:
EMP_kibolet_by_taz=gpd.overlay(taz[['Taz_num','taz_area','geometry']],EMP_kibolet[['ID','emp_area','geometry']])

EMP_kibolet_by_taz['small_area']=EMP_kibolet_by_taz.area

EMP_kibolet_by_taz['pre_small_area_emp']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['emp_area']

EMP_kibolet_by_taz['pre_small_area_taz']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['taz_area']

EMP_kibolet_by_taz=EMP_kibolet_by_taz.query('(pre_small_area_taz >0.7) | (pre_small_area_emp >0.1)')[['ID','Taz_num','geometry']]

EMP_kibolet_by_taz['small_area']=EMP_kibolet_by_taz.area

EMP_kibolet_by_taz=EMP_kibolet_by_taz.set_index('ID')

EMP_kibolet_by_taz['id_area_for_pre']=drop_geo(EMP_kibolet_by_taz).groupby(by='ID').sum()['small_area']

EMP_kibolet_by_taz['prec_from_id']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['id_area_for_pre']

C:\Users\dpere\AppData\Local\Temp\ipykernel_7448\1124101454.py:1: UserWarning: `keep_geom_type=True` in overlay resulted in 279 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  EMP_kibolet_by_taz=gpd.overlay(taz[['Taz_num','taz_area','geometry']],EMP_kibolet[['ID','emp_area','geometry']])


In [270]:
EMP_kibolet['geometry_buff']=EMP_kibolet.buffer(250)

In [271]:
city_muni=['מודיעין עילית','בית שמש','ירושלים','מודיעין - מכבים - רעות']

okev_factor=up_load_df(r'{}\background_files'.format(software_data_folder_location),'okev_factor').set_index('sector')

In [272]:
work_factor=up_load_df(r'{}\background_files'.format(software_data_folder_location),'work_factor_230719').set_index('sector')

## התפלגות אזורים התפלגות ענפי תעסוקה

In [273]:
emp_category_type=up_load_shp(r'{}\background_files\emp_category_type.shp'.format(software_data_folder_location)).fillna(0)

col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

emp_category_type[col]=emp_category_type[col]/100

emp_category_type=gpd.sjoin(make_point(taz.reset_index())[['Taz_num','centroid']],emp_category_type).set_index('Taz_num')

## הכנה ללופ

In [274]:
commuting=up_load_shp(r'{}\background_files\commuting_230712.shp'.format(software_data_folder_location))

In [275]:
pre_woman=0.5

pre_man=1-pre_woman

work_age=[ 'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_50',
'pop_55',
'pop_60']

under_work_age=[ 'pop_15', 'pop_20']

over_work_age=[ 'pop_65', 'pop_70', 'pop_75up']

In [276]:
 #אלו השדות שאולי אני יוריד כאן נכנס המצב הנוכחי
col=['Taz_num',
     'Taz_name',
     'Muni_Heb',
     'jeru_metro',
     'zonetype',
     'in_jerusal',
     'main_secto','jew',
     'student_dorms',
     'student_yeshiva',
     'student',
     'uni_students','emp_from_uni_student','emp_from_Yeshiva_student','yeshiva_dorms_pop_15',
     'yeshiva_dorms_pop_20',
     'yeshiva_dorms_pop_25',
     'yeshiva_dorms_pop_sum',]

forecast_current=forecast_2020[col]

In [277]:
df

,Taz_num,ID,prec_from_id,name,kibolt,realy,note,Shape_Length,Shape_Area,kayim_emp,Palestinians,in_jeru,name_1,F2025,F2030,F2035,F2040,F2045,F2050,geometry,emp_kayim_no_palestinians,emp_current,emp_area,geometry_buff
0,5702.0,46,1.000000,קניון רמות,3000,1500,0,1866.082562,1.015901e+05,750,0.0,1,קניון רמות,975.0,1200.0,1650.0,2100.0,2550.0,3000.0,"MULTIPOLYGON (((218490.407 635839.702, 218495....",750.0,2146.265824,1.015901e+05,"POLYGON ((218217.142 636008.888, 218210.291 63..."
1,5002.0,0,0.286609,הר חוצבים,40000,20000,לבדוק לפי אידנקס,2634.811097,4.989369e+05,12000,0.0,1,הר חוצבים,14800.0,17600.0,23200.0,28800.0,37200.0,40000.0,"MULTIPOLYGON (((219722.072 634811.375, 220254....",12000.0,8559.378488,4.989369e+05,"POLYGON ((219547.390 634990.221, 219565.697 63..."
2,5001.0,0,0.713391,הר חוצבים,40000,20000,לבדוק לפי אידנקס,2634.811097,4.989369e+05,12000,0.0,1,הר חוצבים,14800.0,17600.0,23200.0,28800.0,37200.0,40000.0,"MULTIPOLYGON (((219722.072 634811.375, 220254....",12000.0,21304.896110,4.989369e+05,"POLYGON ((219547.390 634990.221, 219565.697 63..."
3,209.0,22,1.000000,בתי מלון מער מזרח,2000,2000,0,1271.203555,7.877351e+04,1200,0.0,1,בתי מלון מער מזרח,1280.0,1360.0,1520.0,1680.0,1840.0,2000.0,"MULTIPOLYGON (((221539.348 633221.767, 221600....",1200.0,1696.450071,7.877351e+04,"POLYGON ((221289.793 633206.849, 221289.512 63..."
4,5408.0,2,0.137169,גבעת שאול,35000,25000,0,6985.292572,1.077384e+06,18500,0.0,1,גבעת שאול,20150.0,21800.0,25100.0,28400.0,31700.0,35000.0,"MULTIPOLYGON (((217414.313 633103.076, 217426....",18500.0,3942.138038,1.077384e+06,"POLYGON ((216874.554 632753.935, 216875.280 63..."
5,5407.0,2,0.130541,גבעת שאול,35000,25000,0,6985.292572,1.077384e+06,18500,0.0,1,גבעת שאול,20150.0,21800.0,25100.0,28400.0,31700.0,35000.0,"MULTIPOLYGON (((217414.313 633103.076, 217426....",18500.0,3751.654349,1.077384e+06,"POLYGON ((216874.554 632753.935, 216875.280 63..."
6,5405.0,2,0.129964,גבעת שאול,35000,25000,0,6985.292572,1.077384e+06,18500,0.0,1,גבעת שאול,20150.0,21800.0,25100.0,28400.0,31700.0,35000.0,"MULTIPOLYGON (((217414.313 633103.076, 217426....",18500.0,3735.059910,1.077384e+06,"POLYGON ((216874.554 632753.935, 216875.280 63..."
7,5401.0,2,0.178469,גבעת שאול,35000,25000,0,6985.292572,1.077384e+06,18500,0.0,1,גבעת שאול,20150.0,21800.0,25100.0,28400.0,31700.0,35000.0,"MULTIPOLYGON (((217414.313 633103.076, 217426....",18500.0,5129.057407,1.077384e+06,"POLYGON ((216874.554 632753.935, 216875.280 63..."
8,5404.0,2,0.155709,גבעת שאול,35000,25000,0,6985.292572,1.077384e+06,18500,0.0,1,גבעת שאול,20150.0,21800.0,25100.0,28400.0,31700.0,35000.0,"MULTIPOLYGON (((217414.313 633103.076, 217426....",18500.0,4474.974858,1.077384e+06,"POLYGON ((216874.554 632753.935, 216875.280 63..."
9,5403.0,2,0.159925,גבעת שאול,35000,25000,0,6985.292572,1.077384e+06,18500,0.0,1,גבעת שאול,20150.0,21800.0,25100.0,28400.0,31700.0,35000.0,"MULTIPOLYGON (((217414.313 633103.076, 217426....",18500.0,4596.129833,1.077384e+06,"POLYGON ((216874.554 632753.935, 216875.280 63..."


## לופ

In [278]:
year=['2025','2030','2035','2040','2045','2050']

for y in year:
    # חיבור מאפייני אזורי התנועה משכבת אזורי תנועה מצב קיים

   
    print('---------------------------')
    print(y)

    forecast=pd.merge(forecast_current,locals()['forecast_pop_{}_bau'.format(y)],on="Taz_num",how='right')

    
    
    forecast['student_demand']=forecast['pop_0']/5*3*0.5+forecast['pop_0']/5*2+forecast['pop_5']+forecast['pop_10']+forecast['pop_15']/5*3+forecast['pop_0']/5*3*0.5

    demad_for_students_by_muni_and_sector=forecast.pivot_table(index=('Muni_Heb','jew'),aggfunc=sum)[['student_demand','student']]

    demad_for_students_by_muni_and_sector['add_students_to_kayim_from_demand']=demad_for_students_by_muni_and_sector['student_demand']-demad_for_students_by_muni_and_sector['student']
    demad_for_students_by_muni_and_sector.loc[demad_for_students_by_muni_and_sector['add_students_to_kayim_from_demand']<0,'add_students_to_kayim_from_demand']=0

    demad_for_students_by_muni_and_sector=demad_for_students_by_muni_and_sector.reset_index()

    col=['Muni_Heb','jew','add_students_to_kayim_from_demand']

    forecast=forecast.merge(demad_for_students_by_muni_and_sector[col],on=['Muni_Heb','jew'])

    
    forecast['student_to_fill_demand']=forecast['student_demand']-forecast['student']

    forecast.loc[forecast['student_to_fill_demand']<0,'student_to_fill_demand']=0

    forecast=forecast.merge(forecast.groupby(by=['Muni_Heb','jew']).sum()[['student_to_fill_demand']].reset_index(),on=['Muni_Heb','jew'],suffixes=('','_sum'))

    forecast['student_to_fill_demand_pre']=forecast['student_to_fill_demand']/forecast['student_to_fill_demand_sum']

    forecast['add_to_student_current']=forecast['student_to_fill_demand_pre']*forecast['add_students_to_kayim_from_demand']
    
    print_before('student')
    
    forecast=forecast.fillna(0)
    
    forecast['student']=forecast['student']+forecast['add_to_student_current']
    
    print_after('student')

    
    
    
    ## תעסוקה בעקבות תלמידים 

    emp_education_per_student=7

    
    forecast['emp_from_student']=forecast['student']/emp_education_per_student

    print_after('emp_from_student')

    
    
    
    ## סטודנטים 

    forecast['uni_students_pre']=forecast['uni_students']/forecast['uni_students'].sum().item()

    uni_students_pre_from_age_20_30=0.3

    uni_students=forecast.query('main_secto!="Palestinian"')[['pop_20','pop_25']].sum().sum()*uni_students_pre_from_age_20_30
    
    dorms_vs_uni_students=(forecast['student_dorms'].sum().item())/(forecast['uni_students'].sum().item())
    
    print("dorms_vs_uni_students: ",round(dorms_vs_uni_students*100,0), "%")

    
    
    print_before("uni_students")
    
    
    forecast['add_uni_students']=forecast['uni_students_pre']*uni_students-forecast['uni_students']
    
    forecast['uni_students']=forecast['uni_students']+forecast['add_uni_students']
    
    print_after("uni_students")
    
    growth_dorms=((forecast['uni_students'].sum().item())*dorms_vs_uni_students)/(forecast['student_dorms'].sum().item())
    
    print("growth_dorms: ",growth_dorms)
    
    print_before("student_dorms")
    forecast['student_dorms']=forecast['student_dorms']*growth_dorms
    
    print_after("student_dorms")

    ## תעסוקה בעקבות סטודנטים 

    emp_Education_per_uni_student=0.18
    
    print_before('emp_from_uni_student')
    forecast['emp_from_uni_student']=forecast['add_uni_students']*emp_Education_per_uni_student+forecast['emp_from_uni_student']

    print_after('emp_from_uni_student')
   

 
    ## תלמידי ישיבה 

    student_yeshiva_pre_from_age_20_30=0.52
    
    
    print("student_yeshiva before: ",forecast['student_yeshiva'].sum()) 

    

    total_student_yeshiva=forecast.query('jew==1')[['pop_20','pop_25']].sum().sum()*student_yeshiva_pre_from_age_20_30

    print("student_yeshiva after: ",total_student_yeshiva)

    

    forecast['student_yeshiva_pre']=forecast['student_yeshiva']/forecast['student_yeshiva'].sum().item()
    
    ## תעסוקה בעקבות ישיבה 
    
    forecast['student_yeshiva_vs_yeshiva_emp']=forecast['student_yeshiva']/forecast['emp_from_Yeshiva_student']
    
    forecast['student_yeshiva_growth']=(forecast['student_yeshiva_pre']*total_student_yeshiva)/forecast['student_yeshiva']
    
    forecast['student_yeshiva']=forecast['student_yeshiva_pre']*total_student_yeshiva
    
    print("student_yeshiva after: ",forecast['student_yeshiva'].sum())

    print("emp_from_Yeshiva_student before: ",forecast['emp_from_Yeshiva_student'].sum()) 

    forecast['emp_from_Yeshiva_student']=forecast['student_yeshiva']/forecast['student_yeshiva_vs_yeshiva_emp']
    
    print("emp_from_Yeshiva_student after: ",forecast['emp_from_Yeshiva_student'].sum())

   
    ##תוספת בפנימיות בישיבות
    print_before('yeshiva_dorms_pop_sum')

    forecast['yeshiva_dorms_pop_15']=forecast['yeshiva_dorms_pop_15']*forecast['student_yeshiva_growth']
    forecast['yeshiva_dorms_pop_20']=forecast['yeshiva_dorms_pop_20']*forecast['student_yeshiva_growth']
    forecast['yeshiva_dorms_pop_25']=forecast['yeshiva_dorms_pop_25']*forecast['student_yeshiva_growth']
    col_dorms_yeshiva=['yeshiva_dorms_pop_15', 'yeshiva_dorms_pop_20', 'yeshiva_dorms_pop_25']
    forecast['yeshiva_dorms_pop_sum']=forecast[col_dorms_yeshiva].sum(axis=1)
    
    print_after('yeshiva_dorms_pop_sum')

    ## כוללים 

    forecast['kollim_demand']=(forecast['pop_20']*0.8+forecast['pop_25']*0.65+forecast['pop_30']*0.30+forecast['pop_35']*0.30+forecast['pop_40']*0.30+forecast['pop_45']*0.20+forecast['pop_50']*0.20+forecast['pop_55']*0.20+forecast['pop_60']*0.20)*0.5 #הכפלה בחצי בשביל לקבל אוכלוסיית גברים מעורכת

    forecast.loc[forecast['main_secto']!='U_Orthodox','kollim_demand']=0  #אל אף שאנחנו יודעים שיש כוללים באזורים שהם לא מוגדרים כחרדים

    forecast['add_from_kollim_demand']=0

    forecast.loc[(forecast['main_secto']=='U_Orthodox')&(forecast['kollim_demand']>forecast['student_yeshiva']),'add_from_kollim_demand']=forecast['kollim_demand']-forecast['student_yeshiva']

    forecast['student_yeshiva_and_kollim']=forecast['add_from_kollim_demand']+forecast['student_yeshiva']
    
    
    forecast=forecast.fillna(0)



    ## תעסוקה סך חינוך 

    forecast['emp_Education']=forecast['emp_from_student']+forecast['emp_from_Yeshiva_student']+forecast['emp_from_uni_student']
    
    print_after('emp_Education')

    emp_Education=forecast.query('main_secto!="Palestinian"')['emp_Education'].sum().item()
    
    print("emp_Education: ",round(emp_Education,0))

    
    #אזורי תנועה שלא צריכים עוקב
    
    

    EMP_buffer=EMP_kibolet.set_geometry('geometry_buff').query('F{} > 0'.format(y))

    EMP_buffer=EMP_buffer.dissolve()[['geometry_buff']]

    emp_buffer_taz=gpd.overlay(EMP_buffer,taz)

    emp_buffer_taz['emp_samll_area']=emp_buffer_taz.area

    emp_buffer_taz['emp_pre_from_taz']=emp_buffer_taz['emp_samll_area']/emp_buffer_taz['taz_area']

    taz_num_no_need_okev=emp_buffer_taz.loc[emp_buffer_taz['emp_pre_from_taz']>0.4].Taz_num.to_list()
    
    
    forecast['okev']=0
    
    forecast.loc[(~forecast['Taz_num'].isin(taz_num_no_need_okev))&(taz['main_secto']!="Palestinia"),'okev']=1

    forecast.loc[forecast['main_secto']=="arabs_behined_seperation_wall",'okev']=1
    
    
    ## יצירת תעסוקה כנגזר מעוקב

    
    forecast['emp_okev']=0

    forecast.loc[(forecast['jew']==0)&(forecast['okev']==1),'emp_okev']=forecast['aprt']*okev_factor.loc['arab','city']

    forecast.loc[(forecast['main_secto']=="U_Orthodox")&(~forecast['Muni_Heb'].isin(city_muni))&(forecast['okev']==1),'emp_okev']=forecast['aprt']*okev_factor.loc['U_Orthodox','sub']

    forecast.loc[(forecast['main_secto']=="U_Orthodox")&(forecast['Muni_Heb'].isin(city_muni))&(forecast['okev']==1),'emp_okev']=forecast['aprt']*okev_factor.loc['U_Orthodox','city']

    forecast.loc[(forecast['main_secto']=="Jewish")&(forecast['Muni_Heb'].isin(city_muni))&(forecast['okev']==1),'emp_okev']=forecast['aprt']*okev_factor.loc['Jewish','city']

    forecast.loc[(forecast['main_secto']=="Jewish")&(~forecast['Muni_Heb'].isin(city_muni))&(forecast['okev']==1),'emp_okev']=forecast['aprt']*okev_factor.loc['Jewish','sub']

    emp_okev=forecast['emp_okev'].sum().item()
    
    print("emp_okev: ",round(emp_okev,0))
    
    ##  הוספת סטודנטים  לאוכלוסיה ולהתפלגות גילים והוספת תלמידי ישיבה (באזורים חרדיים)

    ## הוספת סטודנטים לאוכלוסיה ולהתפלגות גילים

    forecast=forecast.fillna(0)

    forecast['pop_without_dorms_yeshiva']=forecast['pop']

    forecast['pop']=forecast['pop']+forecast['student_dorms']

    forecast['pop_20_just_from_aprt']=forecast['pop_20']

    forecast['pop_25_just_from_aprt']=forecast['pop_25']

    forecast['pop_20']=forecast['pop_20']+forecast['student_dorms']*0.6

    forecast['pop_25']=forecast['pop_25']+forecast['student_dorms']*0.4

    ## הוספת תלמידי ישיבה (באזורים חרדיים) אל כמות אוכלוסיה,התפלגות גילים 

    forecast['pop_15_just_from_aprt']=forecast['pop_15']
    
    
    forecast.loc[forecast['main_secto']=="U_Orthodox",'pop']=forecast['pop']+forecast['yeshiva_dorms_pop_sum']

    forecast.loc[forecast['main_secto']=="U_Orthodox",'pop_15']=forecast['pop_15']+forecast['yeshiva_dorms_pop_15']

    forecast.loc[forecast['main_secto']=="U_Orthodox",'pop_20']=forecast['pop_20']+forecast['yeshiva_dorms_pop_20']
    
    forecast.loc[forecast['main_secto']=="U_Orthodox",'pop_25']=forecast['pop_25']+forecast['yeshiva_dorms_pop_25']

    ## הוספת סטודנטים אל כמות משקי בית

    forecast=forecast.fillna(0)

    forecast['hh']=forecast['aprt']+forecast['student_dorms']/1.5

    forecast['hh']=forecast['hh']+forecast['yeshiva_dorms_pop_sum']
    
   
    ## יצירת מסגרת מועסקים למרחב



    forecast['work_age']=forecast[work_age].sum(axis=1)

    forecast['under_work_age']=forecast[under_work_age].sum(axis=1)

    forecast['over_work_age']=forecast[over_work_age].sum(axis=1)

    sector=['U_Orthodox','Jewish','Arab','arabs_behined_seperation_wall']
    
    for s in sector:
        
        work_age_factor_woman=work_factor.loc[(work_factor.index==s) & (work_factor['age']=='work_age') & (work_factor['gender']=='woman')& (work_factor['year']==int(y)),'value'].item()
        work_age_factor_man=work_factor.loc[(work_factor.index==s) & (work_factor['age']=='work_age') & (work_factor['gender']=='man')& (work_factor['year']==int(y)),'value'].item()
        under_work_age_factor_woman=work_factor.loc[(work_factor.index==s) & (work_factor['age']=='under_work_age') & (work_factor['gender']=='woman')& (work_factor['year']==int(y)),'value'].item()
        under_work_age_factor_man=work_factor.loc[(work_factor.index==s) & (work_factor['age']=='under_work_age') & (work_factor['gender']=='man')& (work_factor['year']==int(y)),'value'].item()
        over_work_age_factor_woman=work_factor.loc[(work_factor.index==s) & (work_factor['age']=='over_work_age') & (work_factor['gender']=='woman')& (work_factor['year']==int(y)),'value'].item()
        over_work_age_factor_man=work_factor.loc[(work_factor.index==s) & (work_factor['age']=='over_work_age') & (work_factor['gender']=='man')& (work_factor['year']==int(y)),'value'].item()
        
        
        forecast.loc[forecast['main_secto']==s,'pop_emp']=forecast['work_age']*pre_woman*work_age_factor_woman+forecast['work_age']*pre_man*work_age_factor_man
        forecast.loc[forecast['main_secto']==s,'pop_emp']=forecast['pop_emp']+forecast['under_work_age']*pre_woman*under_work_age_factor_woman+forecast['under_work_age']*pre_man*under_work_age_factor_man
        forecast.loc[forecast['main_secto']==s,'pop_emp']=forecast['pop_emp']+forecast['over_work_age']*pre_woman*over_work_age_factor_woman+forecast['over_work_age']*pre_man*over_work_age_factor_man

    

    
        
    ## חישוב אבטלה

    forecast['pop_emp_employed']=0

    forecast.loc[forecast['main_secto']=="U_Orthodox",'pop_emp_employed']=forecast['pop_emp']*0.96

    forecast.loc[forecast['main_secto']=="Jewish",'pop_emp_employed']=forecast['pop_emp']*0.97

    forecast.loc[forecast['jew']==0,'pop_emp_employed']=forecast['pop_emp']*0.96

    ## חישוב יוממות


    forecast=forecast.set_index('Taz_num')

    forecast['commuting']=gpd.sjoin(make_point(taz)[['Taz_num','centroid']],commuting[['commuting', 'geometry']]).set_index('Taz_num')['commuting']/100

    forecast['pop_emp_employed_out_of_jtmt_area']=forecast['pop_emp_employed']*forecast['commuting']

    emp_from_jtmt_area=forecast['pop_emp_employed'].sum().item()-forecast['pop_emp_employed_out_of_jtmt_area'].sum().item()

    emp_in_jtmt_area=emp_from_jtmt_area*1.06 #יוממות פנימה מחוץ למרחב

    emp_in_jtmt_area_without_mobile=emp_in_jtmt_area*0.94 #הפחחת עובדים ניידים
    
    
    print("emp_in_jtmt_area_without_mobile: ",round(emp_in_jtmt_area_without_mobile,0))
    
    ## יצירת תעסוקה לא עוקב

    

    add_emp_not_okev=emp_in_jtmt_area_without_mobile-emp_Education-emp_okev-emp_not_okev_current
    
    print("add_emp_not_okev: ",round(add_emp_not_okev,0))
    
    print("emp_not_okev_current before: ",round(emp_not_okev_current,0))
    
    print("emp_not_okev_current that should be: ",round(emp_not_okev_current+add_emp_not_okev,0))
    

    
    
    add_emp_not_okev_kibolet=EMP_kibolet['F{}'.format(y)].sum().item()-emp_not_okev_current
    
    
    
    print("add_emp_not_okev_kibolet: ",round(add_emp_not_okev_kibolet,0))
    
    add_emp_not_okev_filling=add_emp_not_okev/add_emp_not_okev_kibolet
    
    print("add_emp_not_okev_filling: ",round(add_emp_not_okev_filling,2))

    
    
    EMP_kibolet['emp_current']=(EMP_kibolet['F{}'.format(y)]-EMP_kibolet['emp_current'])*add_emp_not_okev_filling+EMP_kibolet['emp_current']
    
    
    
    emp_not_okev_current=EMP_kibolet['emp_current'].sum().item()
    
    print("emp_not_okev_current after: ",round(emp_not_okev_current,0))

    EMP_kibolet.to_excel(r'{}\Intermediates\EMP_kibolet_{}_for_review_{}.xlsx'.format(software_data_folder_location,y,file_date))
    
    col=['Taz_num','ID','prec_from_id']

    df=EMP_kibolet_by_taz.reset_index()[col].merge(EMP_kibolet,on='ID',how='left')

    df['emp_current']=df['emp_current']*df['prec_from_id']

    forecast['emp_not_okev']=df.drop(columns=['geometry', 'geometry_buff']).pivot_table(index='Taz_num',aggfunc=sum)[['emp_current']]
    
    
    
    print("emp_not_okev_current after coversion to taz: ",round(forecast['emp_not_okev'].sum().item(),0))

    

    forecast=forecast.fillna(0)

    forecast['total_emp']=forecast['emp_not_okev']+forecast['emp_okev']+forecast['emp_Education']


    
    #הסיבה להפרדה כי המקדם השתתפות בעבודה באזורים לא חרדיים גבוהה מידי בשביל התלמידי ישיבה
    forecast.loc[forecast['main_secto']!="U_Orthodox",'pop']=forecast['pop']+forecast['yeshiva_dorms_pop_sum']

    forecast.loc[forecast['main_secto']!="U_Orthodox",'pop_15']=forecast['pop_15']+forecast['yeshiva_dorms_pop_15']

    forecast.loc[forecast['main_secto']!="U_Orthodox",'pop_20']=forecast['pop_20']+forecast['yeshiva_dorms_pop_20']
    
    forecast.loc[forecast['main_secto']!="U_Orthodox",'pop_25']=forecast['pop_25']+forecast['yeshiva_dorms_pop_25']
    
    
    
    col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

    forecast=forecast.join(emp_category_type[col]).fillna(0)

    for i in col:
        forecast['{}'.format(i)]=forecast['{}'.format(i)]*(forecast['emp_not_okev']+forecast['emp_okev'])
        
    forecast['yosh']=0

    forecast.loc[forecast['zonetype']=='Judea and Samaria','yosh']=1

    forecast['jerusalem_city']=0

    forecast.loc[forecast['in_jerusal']=='yes','jerusalem_city']=1

    forecast=forecast.reset_index()
    
    col=['Taz_num',
     'Taz_name',
     'Muni_Heb',
     'jeru_metro',
     'zonetype',
     'in_jerusal',
     'main_secto','jew',
     'student_dorms',
     'student_yeshiva',
     'student',
     'uni_students','emp_from_uni_student','emp_from_Yeshiva_student','yeshiva_dorms_pop_15',
     'yeshiva_dorms_pop_20',
     'yeshiva_dorms_pop_25',
     'yeshiva_dorms_pop_sum']
    
    forecast_current=forecast[col]
    
    
       ## ייצוא תחזית בפורמט

    col=['Taz_num',
    'yosh',
    'jeru_metro',
    'jerusalem_city',
    'main_secto',
    'hh',
    'pop',
    'pop_0',
    'pop_5',
    'pop_10',
    'pop_15',
    'pop_20',
    'pop_25',
    'pop_30',
    'pop_35',
    'pop_40',
    'pop_45',
    'pop_50',
    'pop_55',
    'pop_60',
    'pop_65',
    'pop_70',
    'pop_75up',
    'total_emp',
    'Indus',
    'Com_hotel',
    'Business',
    'Public',
    'emp_Education',
    'agri',
    'student',
    'uni_students',
    'student_yeshiva_and_kollim',
    'pop_emp_employed']

    col_new_name=['TAZ',
    'yosh',
    'in_jerusalem_metropolin',
    'jerusalem_city',
    'sector',
    'hh_total',
    'pop',
    'age0_4',
    'age5_9',
    'age10_14',
    'age15_19',
    'age20_24',
    'age25_29',
    'age30_34',
    'age35_39',
    'age40_44',
    'age45_49',
    'age50_54',
    'age55_59',
    'age60_64',
    'age65_69',
    'age70_74',
    'age75up',
    'emp_tot',
    'indus',
    'com_hotel',
    'business',
    'public',
    'education',
    'agri',
    'student',
    'univ',
    'UO_Hi_Ed',
    'pop_emp_employed']

    forecast_for_model=forecast[col]

    forecast_for_model.columns=col_new_name
    
    save_file_path=r'{}\Intermediates'.format(software_data_folder_location)
    
    save_excel_path=r'{}\{}_forecast_{}.xlsx'.format(save_file_path,file_date,y)

    forecast_for_model.to_excel(save_excel_path,index=False)
    
    logic_test_for_forecast(forecast_for_model)
    
    save_excel_path=r'{}\{}_forecast_{}_full.xlsx'.format(save_file_path,file_date,y)

    forecast.to_excel(save_excel_path,index=False)

---------------------------
2025
student before:  714557.0
student after:  835048.0
emp_from_student after:  119293.0
dorms_vs_uni_students:  12.0 %
uni_students before:  81185.0
uni_students after:  93918.0
growth_dorms:  1.1568497070271264
student_dorms before:  9345
student_dorms after:  10811.0
emp_from_uni_student before:  16056.0
emp_from_uni_student after:  18348.0
student_yeshiva before:  105180
student_yeshiva after:  121648.79640386355
student_yeshiva after:  121648.79640386355
emp_from_Yeshiva_student before:  3903
emp_from_Yeshiva_student after:  4514.121053092598
yeshiva_dorms_pop_sum before:  35558
yeshiva_dorms_pop_sum after:  41126.0
emp_Education after:  142155.0
emp_Education:  142155.0


C:\Users\dpere\AppData\Local\Temp\ipykernel_7448\904272972.py:16: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  demad_for_students_by_muni_and_sector=forecast.pivot_table(index=('Muni_Heb','jew'),aggfunc=sum)[['student_demand','student']]
C:\Users\dpere\AppData\Local\Temp\ipykernel_7448\904272972.py:157: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 248.6774571   290.53924516  241.46713837  143.82066043  202.88854695
   50.66789127   41.80531597   93.33748697  116.28544541  180.10708199
   34.41422708  171.77766527  322.55187373  408.50689821   65.74319113
  177.36920231   69.2045818   102.68291002  200.71068627  249.71329881
  173.78282415  190.72906579  190.18370096   89.97353544   54.572907
   23.95259313  245.19059599  193

emp_okev:  182268.0
emp_in_jtmt_area_without_mobile:  528104.0
add_emp_not_okev:  17311.0
emp_not_okev_current before:  186370.0
emp_not_okev_current that should be:  203681.0
add_emp_not_okev_kibolet:  35131.0
add_emp_not_okev_filling:  0.49
emp_not_okev_current after:  203681.0
emp_not_okev_current after coversion to taz:  203681.0
taz num under 0: []
taz num duplicate: []
yosh_unique: [0, 1]
in_jerusalem_metropolin_unique: [1, 0]
jerusalem_city_unique: [1, 0]
sector_unique: ['arabs_behined_seperation_wall', 'Arab', 'U_Orthodox', 'Jewish', 'Palestinian']
taz num with minus: []
pop more then hh: []
hh missing pop: []
pop missing hh: []
pop vs age dis Mistake: []
taz num with age dis prob: []
taz num with emp total prob: []
taz num with emp dis prob: []
taz num with pop_emp_employed worng : []


C:\Users\dpere\AppData\Local\Temp\ipykernel_7448\2235942893.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taz['pop_check']=taz[col].sum(axis=1)-taz['pop']
C:\Users\dpere\AppData\Local\Temp\ipykernel_7448\2235942893.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taz['pop_check']=taz['pop_check'].round(0)
C:\Users\dpere\AppData\Local\Temp\ipykernel_7448\2235942893.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

---------------------------
2030
student before:  835048.0
student after:  934055.0
emp_from_student after:  133436.0
dorms_vs_uni_students:  12.0 %
uni_students before:  93918.0
uni_students after:  107998.0
growth_dorms:  1.1499098629614486
student_dorms before:  10811.0
student_dorms after:  12431.0
emp_from_uni_student before:  18348.0
emp_from_uni_student after:  20882.0
student_yeshiva before:  121648.79640386355
student_yeshiva after:  144159.53373390465
student_yeshiva after:  144159.53373390465
emp_from_Yeshiva_student before:  4514.121053092598
emp_from_Yeshiva_student after:  5349.445333365942
yeshiva_dorms_pop_sum before:  41126.0
yeshiva_dorms_pop_sum after:  48736.0
emp_Education after:  159668.0
emp_Education:  159668.0
emp_okev:  199586.0


...

C:\Users\dpere\AppData\Local\Temp\ipykernel_7448\2235942893.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taz['pop_check']=taz[col].sum(axis=1)-taz['pop']
C:\Users\dpere\AppData\Local\Temp\ipykernel_7448\2235942893.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taz['pop_check']=taz['pop_check'].round(0)
C:\Users\dpere\AppData\Local\Temp\ipykernel_7448\2235942893.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

---------------------------
2050
student before:  1318121.0
student after:  1451245.0
emp_from_student after:  207321.0
dorms_vs_uni_students:  12.0 %
uni_students before:  150808.0
uni_students after:  167603.0
growth_dorms:  1.111364005479209
student_dorms before:  17359.0
student_dorms after:  19292.0
emp_from_uni_student before:  28588.0
emp_from_uni_student after:  31611.0
student_yeshiva before:  213031.70873591056
student_yeshiva after:  239322.54148735353
student_yeshiva after:  239322.5414873535
emp_from_Yeshiva_student before:  7905.141273970897
emp_from_Yeshiva_student after:  8880.736636481659
yeshiva_dorms_pop_sum before:  72019.0
yeshiva_dorms_pop_sum after:  80907.0
emp_Education after:  247813.0
emp_Education:  247813.0


C:\Users\dpere\AppData\Local\Temp\ipykernel_7448\904272972.py:16: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  demad_for_students_by_muni_and_sector=forecast.pivot_table(index=('Muni_Heb','jew'),aggfunc=sum)[['student_demand','student']]
C:\Users\dpere\AppData\Local\Temp\ipykernel_7448\904272972.py:157: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1013.7075178   470.79032724  622.68928477  185.54101486  378.22555769
  103.49388914  478.64558379  561.0099533    56.25472049  428.08543776
  362.69128836  716.26936987  104.09748876  389.88125006   55.3746265
  149.77432475  460.41987501  573.06133969  329.99934259  276.19080424
  414.01637252   60.94835992  442.38083232  332.30212194  191.25446658
  104.91206099  228.57760392  49

emp_okev:  269172.0
emp_in_jtmt_area_without_mobile:  963387.0
add_emp_not_okev:  58056.0
emp_not_okev_current before:  388347.0
emp_not_okev_current that should be:  446403.0
add_emp_not_okev_kibolet:  209953.0
add_emp_not_okev_filling:  0.28
emp_not_okev_current after:  446403.0
emp_not_okev_current after coversion to taz:  446403.0
taz num under 0: []
taz num duplicate: []
yosh_unique: [0, 1]
in_jerusalem_metropolin_unique: [1, 0]
jerusalem_city_unique: [1, 0]
sector_unique: ['arabs_behined_seperation_wall', 'Arab', 'U_Orthodox', 'Jewish', 'Palestinian']
taz num with minus: []
pop more then hh: []
hh missing pop: []
pop missing hh: []
pop vs age dis Mistake: []
taz num with age dis prob: []
taz num with emp total prob: []
taz num with emp dis prob: []
taz num with pop_emp_employed worng : []


C:\Users\dpere\AppData\Local\Temp\ipykernel_7448\2235942893.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taz['pop_check']=taz[col].sum(axis=1)-taz['pop']
C:\Users\dpere\AppData\Local\Temp\ipykernel_7448\2235942893.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taz['pop_check']=taz['pop_check'].round(0)
C:\Users\dpere\AppData\Local\Temp\ipykernel_7448\2235942893.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

## תוצאות